In [1]:
import pandas as pd
import argparse
import random
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, precision_score, recall_score

import torch

# how to using huggingface tokenizer(bert) 
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification

# using monologg/kobert tokenizer and model
from kobert_transformers.tokenization_kobert import KoBertTokenizer
from transformers import BertModel

# huggingface Trainer
from transformers import Trainer
from transformers import TrainingArguments

#Cutomizer encoder
from bert_dataset import TokenCollator
from bert_dataset import TokenDataset


from transformers.data.data_collator import DataCollatorForTokenClassification
from datasets import load_metric

import torch
from torch.utils.data import Dataset


In [2]:
def BIO_tagging(tok_sentence,ne):
  result=['O' for x in range(len(tok_sentence))]
  ne_no=len(ne.keys())
  if ne_no > 0:
    for idx in range(1,ne_no+1):
      ne_dict=ne[idx]
      isbegin=True
      for word_idx, word in enumerate(tok_sentence):
        if '##' in word:
          word=word.replace('##','')
        if '▁' in word:
          word=word.replace('_','')
        if word in ne_dict['form']:
          if isbegin:
            result[word_idx]=str(ne_dict['label'][:2])+'_B'
            isbegin=False
            continue
          elif isbegin==False & (('_B' in result[word_idx-1]) or '_I' in result[word_idx-1]):
            result[word_idx]=str(ne_dict['label'][:2])+'_I'
            continue
  return result

In [3]:
# 데이터셋 정의

class TokenClassificationDataset(Dataset):

    def __init__(self, texts, labels):  # 리스트의 리스트 형태의 데이터 입력
        self.texts = texts
        self.labels = labels
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]

        return {
            'text': text,
            'label': label }  # 딕셔너리 값으로 리스트를 반환

In [4]:
# 데이터셋 만들기
import random

def get_datasets(texts, labels, valid_ratio=.2):

    # define unique labels
    unique_labels = ['O', 'PS_B', 'PS_I', 'LC_B', 'QT_B', 'QT_I', 'CV_B', 'CV_I', 'LC_I', 'OG_B', 'OG_I', 'DT_B', 'AM_B', 'DT_I', 'EV_B', 'EV_I', 'TM_B', 'TM_I', 'FD_B', 'FD_I', 'AM_I', 'AF_B', 'AF_I', 'TI_B', 'TI_I', 'PT_B', 'PT_I', 'TR_B', 'TR_I', 'MT_B', 'MT_I']

    label_to_index = {}
    index_to_label = {}
    for i, label in enumerate(unique_labels):
        label_to_index[label] = i
        index_to_label[i] = label

    # Convert label text to integer value
    tags = []
    for i in range(len(labels)):                                
      tags.append(list(map(label_to_index.get, labels[i]))) 

    # Shuffle before split into train and validation set.
    shuffled = list(zip(texts, tags))
    random.shuffle(shuffled)
    texts = [e[0] for e in shuffled]
    labels = [e[1] for e in shuffled]
    idx = int(len(texts) * (1 - valid_ratio))

    train_dataset = TokenClassificationDataset(texts[:idx], labels[:idx])
    valid_dataset = TokenClassificationDataset(texts[idx:], labels[idx:])

    return train_dataset, valid_dataset, index_to_label

In [5]:
def tokenize_and_align_labels(example):
    tokenized_input = tokenizer(example["text"], truncation=True, is_split_into_words=True)

    word_ids = tokenized_input.word_ids(batch_index=0)
    previous_word_idx = None
    labels = []
    for word_idx in word_ids: 
      if word_idx is None:
        labels.append(-100)
      elif word_idx != previous_word_idx: 
        labels.append(example['label'][word_idx])
      else:
        labels.append(-100)
      previous_word_idx = word_idx

    tokenized_input["labels"] = labels
    return tokenized_input 

In [6]:
df = pd.read_csv('/content/drive/MyDrive/NER/data/train.tsv', sep='\t')

In [7]:
sentences=df['sentence']
labels=df['label'].apply(lambda x:eval(x))

In [26]:
tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
model = AutoModelForTokenClassification.from_pretrained('klue/bert-base',num_labels=32)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the mo

In [9]:
sentence_tok=[]
for sentence in sentences:
  if len(sentence) < 512:
    sentence_tok.append(tokenizer.tokenize(sentence))
  else:
    sentence_tok.append(tokenizer.tokenize(sentence,max_length=len(sentence),truncation=True))
sentence_tok[0]

['늪', '속', '##으로', '깊이', '빠져', '##드', '##는', '그', '자신', '##이다', '.']

In [10]:
bio_sentence=[]
for tok_list,label in zip(sentence_tok,labels):
  try:
    bio_sentence+=[BIO_tagging(tok_list,label)]
  except Exception as e:
    print(e)
    print(tok_list)
    print(label)
    break  

In [ ]:
# from transformers import AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")  
# #Get the values for input_ids, token_type_ids, attention_mask 
# def tokenize_adjust_labels(all_samples_per_split):   
#     tokenized_samples = tokenizer.batch_encode_plus(all_samples_per_split["tokens"], is_split_into_words=True)   
# #tokenized_samples is not a datasets object so this alone won't work with Trainer API, hence map is used    
# # #so the new keys [input_ids, labels (after adjustment)]   
# # #can be added to the datasets dict for each train test validation split   
#     total_adjusted_labels = []   
#     print(len(tokenized_samples["input_ids"]))   
#     for k in range(0, len(tokenized_samples["input_ids"])):     
#         prev_wid = -1     
#         word_ids_list = tokenized_samples.word_ids(batch_index=k)     
#         existing_label_ids = all_samples_per_split["ner_tags"][k]     
#         i = -1     
#         adjusted_label_ids = []         
#         for wid in word_ids_list:       
#             if(wid is None):         
#                 adjusted_label_ids.append(-100)       
#             elif(wid!=prev_wid):         
#                 i = i + 1         
#                 adjusted_label_ids.append(existing_label_ids[i])         
#                 prev_wid = wid       
#             else:         
#                 label_name = label_names[existing_label_ids[i]]

#                 adjusted_label_ids.append(existing_label_ids[i])              
#                 total_adjusted_labels.append(adjusted_label_ids)   
#                 tokenized_samples["labels"] = total_adjusted_labels   
#     return tokenized_samples  
    
# tokenized_dataset = .map(tokenize_adjust_labels, batched=True)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentence_tok, bio_sentence, test_size=0.2)

In [12]:
train_dataset, valid_dataset, index_to_label = get_datasets(X_train, y_train, valid_ratio=.2)

In [13]:
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels)
tokenized_valid_dataset = valid_dataset.map(tokenize_and_align_labels)

In [14]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [29]:
!pip install wandb 

In [20]:
import os 
import wandb 
os.environ["WANDB_API_KEY"]="35d7152411965fd0aac75a8b2c729d8b0cc29399" 
os.environ["WANDB_ENTITY"]="KOREAN_Tokenizing" 
os.environ["WANDB_PROJECT"]="finetune_klue_bert_ner"



In [32]:
# 평가함수 정의
from datasets import load_metric

metric = load_metric("seqeval")

label_names = ['O', 'PS_B', 'PS_I', 'LC_B', 'QT_B', 'QT_I', 'CV_B', 'CV_I', 'LC_I', 'OG_B', 'OG_I', 'DT_B', 'AM_B', 'DT_I', 'EV_B', 'EV_I', 'TM_B', 'TM_I', 'FD_B', 'FD_I', 'AM_I', 'AF_B', 'AF_I', 'TI_B', 'TI_I', 'PT_B', 'PT_I', 'TR_B', 'TR_I', 'MT_B', 'MT_I']

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)

    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    return flattened_results

In [33]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NER/result",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    logging_steps = 1000,
    report_to="wandb",
    run_name = "wnadb_tokenized",
    save_strategy='epoch'
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,  
    eval_dataset=tokenized_valid_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()
wandb.finish()

using `logging_steps` to initialize `eval_steps` to 1000
PyTorch: setting up devices
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 399638
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 174846
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Problem at: /usr/local/lib/python3.7/dist-packages/transformers/integrations.py 596 setup


KeyboardInterrupt: 